In [1]:
#default_exp core

# AutoTimeSeries

> Core implementation of Nixtla SDK, `autotimeseries`.

In [2]:
#hide 
%load_ext autoreload
%autoreload 2

In [3]:
#export
import json
import os
import requests
import sys
import time
from requests.auth import HTTPBasicAuth
from typing import Dict, List, Optional, Union

import boto3
import botocore
import pandas as pd

In [4]:
#hide
import os

from dotenv import load_dotenv

load_dotenv()

True

In [40]:
#export
class AutoTS:
    """Compute time series features at scale.
    Send an email to fede.garza.ramirez@gmail.com to request access.
    
    Parameters
    ----------
    bucket_name: str
        Bucket name.
    api_id: str
        API identifier. 
    api_key: str
        API key.
    aws_access_key_id: str
        AWS ACCESS KEY ID.
    aws_secret_access_key: str
        AWS SECRET ACCESS KEY.
    """
    
    def __init__(self, bucket_name: str,  
                 api_id: str, api_key: str,
                 aws_access_key_id: str,
                 aws_secret_access_key: str) -> 'AutoTS':
        self.bucket_name = bucket_name
        self.invoke_url = f'https://{api_id}.execute-api.us-east-1.amazonaws.com/main'
        self.api_key = api_key
        self.aws_access_key_id = aws_access_key_id
        self.aws_secret_access_key = aws_secret_access_key
    
    def _parse_response(self, response: str) -> Dict:
        """Parses responde."""
        try:
            resp = json.loads(response)
        except Exception as e:
            raise Exception(response)
    
        return resp
    
    def _progress(self, size: float):
        """Progress bar based on size."""
        downloaded = 0

        def progress(chunk):
            nonlocal downloaded
            downloaded += chunk
            done = int(50 * downloaded / size)
            sys.stdout.write(f'\r[{"=" * done}{" " * (50-done)}]')
            sys.stdout.flush()
        
        return progress
    
    def upload_to_s3(self, file: str, force_upload: bool = False) -> str:
        """Uploads file to s3.
        
        Parameters
        ----------
        file: str
            Local file.
        force_upload: bool
            If True forces uploading even if the file already exists.
        """
        if self.aws_access_key_id is None:
            raise Exception(
                'To use `upload_to_s3` you need to pass '
                '`aws_access_key_id` and '
                '`aws_secret_access_key`'
            )
            
        filename = file.split('/')[-1]
        
        s3 = boto3.client(
            's3',
            aws_access_key_id=self.aws_access_key_id,
            aws_secret_access_key=self.aws_secret_access_key
        )
        # Check if exists
        if not force_upload:
            try:
                session = boto3.Session(
                    aws_access_key_id=self.aws_access_key_id,
                    aws_secret_access_key=self.aws_secret_access_key
                )
                
                session.resource('s3').Object(self.bucket_name, filename).load()
            except botocore.exceptions.ClientError as e:
                if e.response['Error']['Code'] != "404":
                    raise e
            else:
                # The object does exist
                return filename
        
        # Progress bar
        size = os.stat(file).st_size
        progress_bar = self._progress(size)
        
        # Uploading file
        s3.upload_file(file, self.bucket_name, filename, Callback=progress_bar)
            
        return filename 
    
    def download_from_s3(self, filename: str, filename_output: Optional[str] = None) -> str:
        """Downloads file from s3.

        Parameters
        ----------
        filename: str
            File to download.
        filename_output: str
            Filename output.
        """
        if self.aws_access_key_id is None:
            raise Exception(
                'To use `download_from_s3` you need to pass '
                '`aws_access_key_id` and '
                '`aws_secret_access_key`'
            )
        
        s3 = boto3.client('s3',
                          aws_access_key_id=self.aws_access_key_id,
                          aws_secret_access_key=self.aws_secret_access_key)
        
        # Progress bar
        meta_data = s3.head_object(Bucket=self.bucket_name, Key=filename)
        size = int(meta_data.get('ContentLength', 0))
        progress_bar = self._progress(size)

        # Downloading file
        s3.download_file(self.bucket_name, filename, 
                         filename if filename_output is None else filename_output, 
                         Callback=progress_bar)
        
    def _tsfeatures(self, filename: str, freq: int, 
                    kind: str = 'static',
                    unique_id_column: str = 'unique_id',
                    ds_column: str = 'ds',
                    y_column: str = 'y') -> Dict:
        """Calculates features from S3 URL.
        
        Parameters
        ----------
        filename: str
            Filename of the dataset.
            The dataset should contain at least three columns:
                - Time series identifier.
                - Time identifier.
                - Target identifier.
        freq: int
            Frequency of the time series.
        kind: str
            Kind of features. Static returns features for each time series.
            temporal returns for each ds and each time series.
        unique_id_column: str
            Column name identifying each time series.
        ds_column: str
            Column name identifying each time stamp.
        y_column: str
            Column name identifying the target variable.
            
        Notes
        -----
        [1] The dataset should contain time series of the same frequency.
        """
        query = dict(
            s3_args=dict(
                s3_url=f's3://{self.bucket_name}',
            ), 
            args = dict(
                filename=filename,
                kind=kind,
                freq=freq,
                unique_id_column=unique_id_column,
                ds_column=ds_column,
                y_column=y_column,
            )
        )

        resp = requests.post(f'{self.invoke_url}/tsfeatures', 
                             headers={'x-api-key': self.api_key},
                             data=json.dumps(query))
        
        return self._parse_response(resp.text)
    
    def tsfeatures_temporal(self, filename: str, freq: int, 
                            unique_id_column: str = 'unique_id',
                            ds_column: str = 'ds',
                            y_column: str = 'y') -> pd.DataFrame:
        """Calculates temporal features from S3 URL.
        
        Parameters
        ----------
        filename: str
            Filename of the dataset.
            The dataset should contain at least three columns:
                - Time series identifier.
                - Time identifier.
                - Target identifier.
        freq: int
            Frequency of the time series.
        unique_id_column: str
            Column name identifying each time series.
        ds_column: str
            Column name identifying each time stamp.
        y_column: str
            Column name identifying the target variable.
            
        Notes
        -----
        [1] The dataset should contain time series of the same frequency.
        """
        
        return self._tsfeatures(s3_uri=s3_uri,
                                kind='temporal',
                                freq=freq,
                                unique_id_column=unique_id_column,
                                ds_column=ds_column,
                                y_column=y_column)
    
    def tsfeatures_static(self, filename: str, freq: int, 
                          unique_id_column: str = 'unique_id',
                          ds_column: str = 'ds',
                          y_column: str = 'y') -> Dict:
        """Calculates static features from S3 URL.
        
        Parameters
        ----------
        filename: str
            Filename of the dataset.
            The dataset should contain at least three columns:
                - Time series identifier.
                - Time identifier.
                - Target identifier.
        freq: int
            Frequency of the time series.
        unique_id_column: str
            Column name identifying each time series.
        ds_column: str
            Column name identifying each time stamp.
        y_column: str
            Column name identifying the target variable.
            
        Notes
        -----
        [1] The dataset should contain time series of the same frequency.
        """
        
        return self._tsfeatures(s3_uri=s3_uri,
                                kind='static',
                                freq=freq,
                                unique_id_column=unique_id_column,
                                ds_column=ds_column,
                                y_column=y_column)
    
    def calendartsfeatures(self, filename: str, country: str, 
                           events: Optional[Union[str, Dict[str, List[str]]]] = None,
                           unique_id_column: str = 'unique_id',
                           ds_column: str = 'ds',
                           y_column: str = 'y') -> Dict:
        """Calculates calendar features from S3 URL.
        
        Parameters
        ----------
        filename: str
            filename of the dataset.
            The dataset should contain at least three columns:
                - Time series identifier.
                - Time identifier.
                - Target identifier.
        country: str
            Country to calculate calendar features from.
        events: Union[str, Dict[str, List[str]]]
            - str file
                String file of events with the format
                event1=fecha1,fecha2/event2=fecha1,fecha2,fecha3
                allocated in the bucket.
            - Dictionary of events.
                The key defines the name of the event and 
                the value is a list of the dates to consider.
                Ej. {'important_event_1': ['2010-02-05', '2011-02-07', '2012-02-08', '2013-02-09'],
                     'important_event_2': ['2010-03-14', '2011-03-12', '2012-03-16', '2013-03-18'],}
                The format of each date should be 'YYYY-MM-DD'.
        unique_id_column: str
            Column name identifying each time series.
        ds_column: str
            Column name identifying each time stamp.
        y_column: str
            Column name identifying the target variable.
        """
        query = dict(
            s3_args=dict(
                s3_url=f's3://{self.bucket_name}',
            ), 
            args = dict(
                filename=filename,
                country=country,
                unique_id_column=unique_id_column,
                ds_column=ds_column,
                y_column=y_column,
            )
        )
        
        if events is not None:
            if isinstance(events, str):
                query['args']['events'] = events
            else:
                str_events = [f'{key}={",".join(value)}' for key, value in events.items()]
                str_events = '/'.join(str_events)
                query['args']['events'] = str_events
        
        resp = requests.post(f'{self.invoke_url}/calendartsfeatures', 
                             headers={'x-api-key': self.api_key},
                             data=json.dumps(query))
        
        return self._parse_response(resp.text)
    
    def tsforecast(self, 
                   filename_target: str, 
                   freq: str,
                   seasonality: int,
                   horizon: int,
                   unique_id_column: str = 'unique_id',
                   ds_column: str = 'ds',
                   y_column: str = 'y') -> Dict:
        """Calculates forecast.
        
        Parameters
        ----------
        filename_target: str
            Filename of the target dataset.
            The dataset should contain at least three columns:
                - Time series identifier.
                - Time identifier.
                - Target identifier.
        freq: str
            Frequency of the data.
            Only 'D' supported.
        horizon: int
            Forcast horizon.
        unique_id_column: str
            Column name identifying each time series.
        ds_column: str
            Column name identifying each time stamp.
        y_column: str
            Column name identifying the target variable.
        """
        query = dict(
            s3_args=dict(
                s3_url=f's3://{self.bucket_name}',
            ), 
            data_args = dict(
                filename=filename_target,
                freq=freq,
                seasonality=seasonality,
                horizon=horizon,
                unique_id_column=unique_id_column,
                ds_column=ds_column,
                y_column=y_column,
            )
        )
        
        resp = requests.post(f'{self.invoke_url}/tsforecast', 
                             headers={'x-api-key': self.api_key},
                             data=json.dumps(query))
        
        return self._parse_response(resp.text)
    
    def tsbenchmarks(self, 
                     filename: str,
                     dataset: str,
                     unique_id_column: str = 'unique_id',
                     ds_column: str = 'ds',
                     y_column: str = 'y') -> Dict:
        """Calculates performance based on filename and dataset.
        
        Parameters
        ----------
        filename: str
            Filename of the dataset.
            The dataset should contain at least three columns:
                - Time series identifier.
                - Time identifier.
                - Target identifier.
        dataset: str
            Target dataset.
        unique_id_column: str
            Column name identifying each time series.
        ds_column: str
            Column name identifying each time stamp.
        y_column: str
            Column name identifying the target variable.
        """
        query = dict(
            s3_args=dict(
                s3_url=f's3://{self.bucket_name}',
            ), 
            args = dict(
                filename=filename,
                dataset=dataset,
                unique_id_column=unique_id_column,
                ds_column=ds_column,
                y_column=y_column,
            )
        )
        
        resp = requests.post(f'{self.invoke_url}/tsbenchmarks', 
                             headers={'x-api-key': self.api_key},
                             data=json.dumps(query))
        
        return self._parse_response(resp.text)
    
    
    def get_status(self, job_id: str) -> Dict:
        """Gets job status.
        
        Parameters
        ----------
        job_id: str
            ID job from requests.
        """
        resp = requests.get(f'{self.invoke_url}/jobs/', 
                            params={'job_id': job_id},
                            headers={'x-api-key': self.api_key})
        
        output = self._parse_response(resp.text)
        
        return output
    
    def check_progress_logs(self, job_id: str, sleep_time: int = 45) -> Dict:
        """Waits untile `job_id` is done and prints logs."""
        sys.stdout.write(
            """
            @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
            @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
            @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
            @@..*@@@@@@@.@@@@@@@@/.@@@@@@@@..@@@@@..@@@@@@..........@@@@@@..@@@@@@@@@@@@@@@@@.../@@@@@@@
            @@.@*.(@@@@@.@@@@@@@@/.@@@@@@@@@@..@..@@@@@@@@@@@@..@@@@@@@@@@..@@@@@@@@@@@@@@@@..@@.,@@@@@@
            @@.@@@,./@@@.@@@@@@@@/.@@@@@@@@@@@...@@@@@@@@@@@@@..@@@@@@@@@@..@@@@@@@@@@@@@@@.*@@@@..@@@@@
            @@.@@@@@/.(@.@@@@@@@@/.@@@@@@@@@*.@@&.,@@@@@@@@@@@..@@@@@@@@@@..@@@@@@@@@@@@@@..........@@@@
            @@.@@@@@@@...@@@@@@@@/.@@@@@@@(./@@@@@..&@@@@@@@@@..@@@@@@@@@@......@@@@@@@@/.@@@@@@@@@,.@@@
            @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
            @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
            
            Have some coffee while you wait...
               ( (
                ) )
              ........
              |      |]
              \      /    
               `----'
            """
        )
        sys.stdout.write(f'\nSetting up Nixtla infrastructure (this will take up 5 mins)...\n')
        
        idx_logs = 0
        in_progress = True
        while in_progress:
            resp = self.get_status(job_id)
            status = resp['status']
            logs = json.loads(resp['logs'])
            
            if status != 'InProgress' and not logs:
                time.sleep(30)
                resp = self.get_status(job_id)
                status = resp['status']
                logs = json.loads(resp['logs'])
            
            if logs:
                #if logs != latest_logs:
                for log in logs[idx_logs:]:
                    sys.stdout.write(f'{log}\n')
                #latest_logs = logs
                idx_logs = len(logs)
            
            in_progress = status == 'InProgress'
            
            time.sleep(sleep_time)
                        
        return status

# Time Series as a Service

In [43]:
autotimeseries = AutoTS(bucket_name=os.environ['BUCKET_NAME'],
                        api_id=os.environ['API_ID'], 
                        api_key=os.environ['API_KEY'],
                        aws_access_key_id=os.environ['AWS_ACCESS_KEY_ID'], 
                        aws_secret_access_key=os.environ['AWS_SECRET_ACCESS_KEY'])

## M5 example

In [29]:
filename_target = autotimeseries.upload_to_s3('../data/target.parquet')
filename_static = autotimeseries.upload_to_s3('../data/static.parquet')
filename_temporal = autotimeseries.upload_to_s3('../data/temporal.parquet')

[==================================================]

In [30]:
print(filename_target, filename_static, filename_temporal, sep='\n')

target-1.parquet
static.parquet
temporal.parquet


Each time series of the uploaded datasets is defined by the column `item_id`. Meanwhile the time column is defined by `timestamp` and the target column by `demand`. We need to pass this arguments to each call.

In [31]:
columns = dict(unique_id_column='item_id',
               ds_column='timestamp',
               y_column='demand')

## Computing calendar features

In [177]:
response_calendar = autotimeseries.calendartsfeatures(filename=filename_temporal,
                                                      country='USA',
                                                      **columns)

In [178]:
response_calendar

{'id_job': '11ab846d-b2b8-42a9-bd91-787668cf292d',
 'dest_url': 's3://nixtla-user-test/calendar-features.csv',
 'status': 200,
 'message': 'Check job status at GET /jobs/{job_id}'}

To check the status of your job use the method `get_status`:

In [179]:
autotimeseries.get_status(response_calendar['id_job'])

{'status': 'InProgress', 'processing_time_seconds': 0, 'logs': '[]'}

If you want to use the result of your job, you have to wait until its status is `Completed`.

## Make forecasts

In [35]:
response_forecast

{'id_job': '736ffa02-763d-4554-bcea-8a62a7081a70',
 'dest_url': 's3://m5example/forecasts.csv',
 'status': 200,
 'message': 'Check job status at GET /jobs/{job_id}'}

## Download forecasts from S3

In [ ]:
autotimeseries.download_from_s3(filename='forecasts.csv', filename_output='../data/forecasts.csv')

[==================================================]

## Evaluate performance 

In [ ]:
response_benchmarks = autotimeseries.tsbenchmarks(filename='forecasts.csv',
                                                  dataset='M5')

In [ ]:
response_benchmarks

{'id_job': '7dbd525e-3fa9-416a-86ef-fbdadda129a3',
 'dest_url': 's3://nixtla-user-test/benchmarks.csv',
 'status': 200,
 'message': 'Check job status at GET /jobs/{job_id}'}

In [ ]:
autotimeseries.get_status(response_benchmarks['id_job'])

Successfully decompressed data/m5/datasets/m5.zip
Successfully downloaded m5.zip, 50219189, bytes.
Decompressing zip file...
Reading file...
File readed.
Computing metrics...


{'status': 'Completed', 'processing_time_seconds': 348}

### Download evaluation

In [ ]:
autotimeseries.download_from_s3(filename='benchmarks.csv', filename_output='../data/benchmarks.csv')